*JUAN CAMILO SERRANO CORREA*

In [67]:
import tensorflow as tf
import keras
#tf.compat.v1.disable_eager_execution ()
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras import backend as k
from keras.preprocessing import image

In [68]:
iv3 = InceptionV3()

In [ ]:
print(iv3.summary())

In [70]:
x = image.img_to_array(image.load_img("/content/lion.jpg", target_size=(299, 299))) #se importa la imagen y se transforma a un Array para ajustar el tamaño de la imagen automaticamente se usa el target_size

x /= 255 #Valores entre 0 y 1
x -= 0.5 #valores entre -0.5 y 0.5
x *= 2 #Valores entre -1 y 1

x = x.reshape([1, x.shape[0], x.shape[1], x.shape[2]])#Se inclucye la dimension del batch

y = iv3.predict(x)
decode_predictions(y)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[[('n02129165', 'lion', 0.9269946),
  ('n04328186', 'stopwatch', 0.0015906388),
  ('n02096294', 'Australian_terrier', 0.0014667814),
  ('n02128385', 'leopard', 0.0010318541),
  ('n02129604', 'tiger', 0.00079344545)]]

##ATAQUES ADVERSARIOS

In [71]:
#Modifica capa de entrada, para que prediga otra cosa, siendo la misma imagen
inp_layer = iv3.layers[0].input
out_layer = iv3.layers[-1].output

target_class =951 #probabilidad de acierto
loss = out_layer[0, target_class]
grad = k.gradients(loss, inp_layer)[0]
optimize_gradient = k.function([inp_layer,k.learning_phase()],[grad, loss])
#Manipulacion de imagen, con cuidado de que los cambios sean visibles
adv = np.copy(x)
pert = 0.001
max_pert = x + 0.001
min_pert = x - 0.001
cost = 0.0
while cost < 0.95:
  gr,cost = optimize_gradient([adv, 0])#Testing mode=0
  adv += gr
  adv = np.clip(adv,min_pert, max_pert)#Imagen dentro de rango de perturbacion
  adv = np.clip(adv, -1, 1)
  print("Lemon cost", cost)
hacked = np.copy(adv)


Lemon cost 8.5595646e-05
Lemon cost 8.578571e-05
Lemon cost 8.597683e-05
Lemon cost 8.6166154e-05
Lemon cost 8.635533e-05
Lemon cost 8.654274e-05
Lemon cost 8.672761e-05
Lemon cost 8.690796e-05
Lemon cost 8.708644e-05
Lemon cost 8.726507e-05
Lemon cost 8.744505e-05
Lemon cost 8.762619e-05
Lemon cost 8.780739e-05
Lemon cost 8.798846e-05
Lemon cost 8.817083e-05
Lemon cost 8.8354216e-05
Lemon cost 8.853908e-05
Lemon cost 8.872419e-05
Lemon cost 8.890839e-05
Lemon cost 8.909228e-05
Lemon cost 8.9278066e-05
Lemon cost 8.946622e-05
Lemon cost 8.965314e-05
Lemon cost 8.983666e-05
Lemon cost 9.00205e-05
Lemon cost 9.020453e-05
Lemon cost 9.038943e-05
Lemon cost 9.0572736e-05
Lemon cost 9.0759e-05
Lemon cost 9.094504e-05
Lemon cost 9.113087e-05
Lemon cost 9.132196e-05
Lemon cost 9.151197e-05
Lemon cost 9.1701564e-05
Lemon cost 9.189039e-05
Lemon cost 9.208019e-05
Lemon cost 9.227139e-05
Lemon cost 9.246347e-05
Lemon cost 9.265994e-05
Lemon cost 9.285628e-05
Lemon cost 9.305385e-05
Lemon cost 9.

KeyboardInterrupt: ignored

In [ ]:
adv /=2
adv += 0.5
adv *= 255

In [ ]:
plt.imshow(adv[0].astype(np.unit8))
plt.show()

In [ ]:
from PIL import Image
im = Image.fromarray(adv[0].astype(np.unit8))
im,save("./hacked.png")#png